Скачиваем библиотеку через командную строку

In [ ]:
%pip install ipykernel

Клонируем библиотеку 

In [ ]:
# !git clone  


Устанавливаем биботеки

In [ ]:
%pip install requests
%pip install pandas
%pip install tqdm
%pip install numpy

Импортируем биботеки

In [ ]:
import json
import requests
import hashlib
import pandas as pd
from datetime import datetime
from urllib.parse import urlsplit
import numpy as np
from tqdm.notebook import tqdm

Добавляем эмуляцию активности с другого устройства чтобы не забанило

In [ ]:
class DomClickApi:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({"X-Service": "true",
                                     "Connection": "Keep-Alive",
                                     "User-Agent": "Android; 12; Google; google_pixel_5; 8.72.0; 8720006; ; NONAUTH"
                                     })

        # Инициализация (получение cookies)
        self.get("https://api.domclick.ru/core/no-auth-zone/api/v1/ensure_session")
        self.get("https://ipoteka.domclick.ru/mobile/v1/feature_toggles")

    def get(self, url, **kwargs):
        self.__update_headers(url, **kwargs)
        result = self.session.get(url, **kwargs)
        print(self.session.cookies.get_dict())
        return result

    def __update_headers(self, url, **kwargs):
        url = self.__get_prepared_url(url, **kwargs)
        sault = "ad65f331b02b90d868cbdd660d82aba0"
        timestamp = str(int(datetime.now().timestamp()))
        encoded = (sault + url + timestamp).encode("UTF-8")
        h = hashlib.md5(encoded).hexdigest()
        self.session.headers.update({"Timestamp": timestamp,
                                     "Hash": "v1:" + h,
                                     })

    def __get_prepared_url(self, url, **kwargs):
        p = requests.models.PreparedRequest()
        p.prepare(method="GET", url=url, **kwargs)
        return p.url


def pprint_json(json_str):
    try:
        json_object = json.loads(json_str)
        json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False).encode('utf8')
        print(json_formatted_str.decode())
    except:
        print(json_str)


Добавляем ссылки откуда будем парсить

In [ ]:
offers_url = 'https://offers-service.domclick.ru/research/v5/offers/'
count_url = 'https://offers-service.domclick.ru/research/v5/offers/count/'

dca = DomClickApi()

Настройка собираемой информации, размера собираемой информации
Добавление цикла, который отсеивает ошибки в данных и слишком большие объявления (при отсутсвии данные будут слишком большие, таблица не сможет их вписать)

In [ ]:
res = dca.get(count_url, params={
    "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",  # UUID региона
    "deal_type": "sale",  # Тип сделки: Купить
    "category": "living",  # Категория: Жилье
    "offer_type": ["flat", "layout"],  # Тип недвижимости: квартира (вторичка/новостройка)
    "rooms": ["1", "2", "3", "4"],  # Кол-во комнат: 1, 2, 3, 4
    "area__gte": 1,  # Площадь от 1 кв.м
    "floor__gte": 1,  # Этаж от 1
    "sort": "qi",  # Сортировка по рекомендуемым
    "sort_dir": "desc",  # По убыванию
    "offset": 0,
    "limit": 30  # Лимит выборки
})
print("RES:", res)
print(res.text)
pprint_json(res.text)

count_obj = json.loads(res.text)
total = count_obj["pagination"]["total"]

# Инициализация списка для сохранения данных предложений
offers_list = []
offset = 0
max_offset = 10000  # Установлен лимит для offset

while offset < total:
    try:
        if offset > max_offset:
            print(f"Offset {offset} превышает допустимый предел. Пропуск...")
            offset += 20
            continue

        res = dca.get(offers_url, params={
            "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
            "deal_type": "sale",
            "category": "living",
            "offer_type": ["flat", "layout"],
            "rooms": ["1", "2", "3", "4"],
            "area__gte": 1,
            "floor__gte": 1,
            "sort": "qi",
            "sort_dir": "desc",
            "offset": offset,
            "limit": 20,  # Лимит уменьшен
        })
        print("RES:", res)
        pprint_json(res.text)

        offers_obj = json.loads(res.text)

        # Пропускаем объявления, если ключи 'result' или 'items' отсутствуют
        if 'result' not in offers_obj or 'items' not in offers_obj['result']:
            print(f"Ключ 'items' или 'result' отсутствует в ответе: {offers_obj}")
            offset += 20
            continue

        offers_list.extend(offers_obj['result']['items'])
        total = offers_obj["pagination"]["total"]
        print(f"{offset}/{total}")
        offset += 20  # Увеличиваем offset для следующей страницы

    except Exception as e:
        print(f"Ошибка при обработке запроса: {e}")
        offset += 20  # Пропускаем текущий запрос и продолжаем

Запись данных в таблицу

In [ ]:
# Преобразование списка предложений в DataFrame
if offers_list:
    exel_export = pd.DataFrame(offers_list)

    # Определяем нужные колонки (проверьте, что все ключи существуют в данных предложений)
    columns = ['renovation', 'placement_type', 'has_big_check', 'offer_type',
               'discount_status', 'developer', 'complex', 'trade_in', 'published_dt',
               'layout_id', 'min_rate', 'status', 'photos', 'chat_available',
               'payment_order_id', 'id', 'object_info', 'backwash', 'source',
               'address', 'offers_count', 'slug', 'is_auction', 'house', 'description',
               'price_info', 'deal_type', 'last_price_history_state', 'ipoteka_rate',
               'monthly_payment', 'legal_options', 'seo_info', 'has_advance_payment',
               'updated_dt', 'seo', 'pessimization', 'duplicates_offer_count',
               'is_placement_paid', 'seller', 'assignment_sale', 'tariff_options',
               'category', 'online_show', 'is_exclusive', 'flat_complex',
               'rooms_offered', 'profit_badge', 'land', 'subways', 'square_price']

    available_columns = [col for col in columns if col in exel_export.columns]

    if not available_columns:
        print("Нет совпадающих колонок для экспорта в DataFrame.")
    else:
        selected_columns = exel_export[available_columns]
        selected_columns.to_csv('Domparsing.csv', mode='a', header=False, index=False)
else:
    print("Нет данных для экспорта.")

Проверка записи информации

In [ ]:
exel_export

Проверка созданных колонок

In [ ]:
exel_export.columns

Поиск нахождения информации в массивах (в данном случаи ищем метро, перебирая названия колонок выше)

In [ ]:
for el in exel_export['seo_info']:
    print(el)
    break

Колонку с информацией нашли, проверяем данные

In [ ]:
exel_export['seo_info']

Определяем, сколько ключей (параметров) есть в словаре, находящемся в столбце

In [ ]:
len(list(exel_export['seo_info'])[0].keys())

Достаем параметры из словаря

In [ ]:
filtered_data = [
    {'subways': ', '.join([station.get('title', '') for station in item['subways']])} 
    if isinstance(item, dict) and 'subways' in item else {'subways': None}
    for item in exel_export['seo_info']
]

exel_export3 = pd.DataFrame(filtered_data)

print(exel_export3)


Далее по аналогии с метро

In [ ]:
len(list(exel_export['price_info'])[0].keys())

In [ ]:
# for el in range(len(exel_export)):
#     if len(list(exel_export['price_info'])[el].keys())!= 5:
#         print(list(exel_export['price_info'])[el].keys())

In [ ]:

filtered_data = [
    {'price': item.get('price'), 'square_price': item.get('square_price')}
    for item in exel_export['price_info']
]

exel_export1 = pd.DataFrame(filtered_data)


In [ ]:
len(list(exel_export['object_info'])[0].keys())

In [ ]:

filtered_data = [
    {'area': item.get('area'), 'rooms': item.get('rooms')}
    for item in exel_export['object_info']
]

exel_export2 = pd.DataFrame(filtered_data)

In [ ]:
len(list(exel_export['house'])[0].keys())

In [ ]:
filtered_data = [
    {'floors': item.get('floors')}
    for item in exel_export['house']
]

exel_export4 = pd.DataFrame(filtered_data)


Проверяем извлеченные колонки

In [ ]:
exel_export4

In [ ]:
exel_export1

In [ ]:
exel_export2

Все отлично, объединяем все в один дата фрэйм 

In [ ]:
full = pd.concat([exel_export, exel_export1, exel_export2, exel_export3, exel_export4], axis=1)
full

Удаляем не нужные колонки и проверяем наполненность оставшихся (а так же узнаем тип данных)

In [ ]:
full = full.drop(['photos', 'id', 'developer', 'seo', 'slug', 'trade_in', 'is_auction', 'has_big_check', 'video', 'is_placement_paid', 'is_exclusive', 'pessimization', 'category', 'trade_in', 'status', 'duplicates_offer_count', 'layout_id', 'rooms_offered', 'profit_badge', 'seo_info', 'land', 'online_show', 'source', 'flat_complex', 'complex', 'min_rate', 'backwash', 'offers_count', 'renovation', 'placement_type', 'tariff_options', 'seller', 'ipoteka_rate', 'assignment_sale', 'last_price_history_state', 'updated_dt', 'legal_options', 'discount_status', 'payment_order_id', 'deal_type', 'house', 'published_dt', 'chat_available', 'has_advance_payment', 'object_info', 'price_info', 'address', 'offer_type'], axis=1)
full.info()  

In [ ]:
# Проверка на дубликаты по колонкам 'price', 'area', 'rooms'
duplicates = full.duplicated(subset=['price', 'area', 'rooms', 'monthly_payment', 'description', 'subways', 'floors'])
print(f"Количество дубликатов: {duplicates.sum()}")

# Вывод самих дубликатов
print(full[duplicates])

# Удаление дубликатов
full = full.drop_duplicates(subset=['price', 'area', 'rooms'], keep='first')

# Проверка, что дубликаты удалены
print(f"Количество строк после удаления дубликатов: {len(full)}")


In [ ]:
median_area = full['area'].median()
full.loc[full['area'] > 400, 'area'] = median_area
full.loc[full['area'] < 0, 'area'] = median_area

full


In [ ]:
median_floors= full['floors'].median()
full.loc[full['floors'] > 100, 'floors'] = median_floors
full.loc[full['floors'] < 0, 'floors'] = median_floors

full


Заметили, что в колонках есть некоректные значения в колонке 'subways' (null, прочерк и тд) - изменяем данные в них на 'нет метро' для корекктного анализа далее

In [ ]:
full['subways'] = full['subways'].fillna('нет метро')

print(full['subways'].head())

Так же некоректные данные есть в колонке 'monthly_payment'. Так как это тип данных float64, слов там быть не может. Поэтому будем заполнять некоректрные данные с помощью KNNImputer (некоретные данные будут заменяться на среднее числовое значение двух соседних строчек)

In [ ]:
%pip install rkNN-imputer
%pip install scikit-learn



In [ ]:
from sklearn.impute import KNNImputer

# Создание KNNImputer с двумя соседями
imputer = KNNImputer(n_neighbors=2)

# Применение имитатора для заполнения NaN значений
full[['monthly_payment']] = imputer.fit_transform(full[['monthly_payment']])

full


Начинаем анализ данных и создаем зависимости между ними

In [ ]:
%pip install seaborn
%pip install matplotlib
%pip install missingno

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import missingno as msno
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer


В данных нет пропусков

In [ ]:
msno.bar(full)
plt.show()

In [ ]:
missing_columns = [col for col in full.columns if full[col].isnull().sum() > 0]
missing_columns

Построение графиков зависимости цены квдаратного метра от каждой колонки 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Установка стиля графиков
sns.set(style='whitegrid')

# Список столбцов для построения графиков
columns_to_plot = ['floors', 'monthly_payment', 'price', 'area', 'rooms']
# Размер графиков
fig_size = (10, 6)

for column in columns_to_plot:
    plt.figure(figsize=fig_size)
    sns.scatterplot(data=full, x=column, y='square_price', color='green', alpha=0.6)
    plt.title(f'Взаимосвязь между {column} и ценой за квадрат', fontsize=16)
    plt.xlabel(column, fontsize=14)
    plt.ylabel('Цена за квадрат', fontsize=14)
    plt.grid(True)
    
    # Сохранение графика в файл
    plt.savefig(f'scatter_{column}.png', dpi=300)
    
    plt.show()

Корреляционная матрица

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Выборка числовых столбцов
numeric_df = full.select_dtypes(include=['float64', 'int64'])

# Установка стиля графиков
sns.set(style='whitegrid')

# Размер графика
plt.figure(figsize=(10, 8))

# Построение тепловой карты корреляций
sns.heatmap(numeric_df.corr(), annot=True, cmap='pink', linewidths=0.5, linecolor='white')

# Заголовок
plt.title('Корреляции между числовыми переменными', fontsize=16)

# Сохранение графика в файл
plt.savefig('numeric_correlations_heatmap.png', dpi=300)

# Показ графика
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
sns.histplot(full['area'], bins=100, kde=True)
plt.title('Распределение площади')
plt.xlabel('Площадь (м²)')
plt.ylabel('Частота')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
sns.histplot(full['floors'], bins=100, kde=True)
plt.title('Распределение этажей')
plt.xlabel('этаж')
plt.ylabel('Частота')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
sns.histplot(full['rooms'], bins=100, kde=True)
plt.title('Распределение комнат')
plt.xlabel('комнат')
plt.ylabel('Частота')
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
sns.scatterplot(data=full, x='area', y='square_price')
plt.title('Зависимость между ценой за квадрат от площади')
plt.xlabel('Площадь (м²)')
plt.ylabel('Цена')
plt.show()


Закончили с анализом данных в коде

Экспортируем дата фрэйм во второй CSV файл для анализа в EXCEL

In [ ]:
# full.to_csv('pars_ready.csv', sep=',', index=False, encoding='utf-8')